In [2]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import matplotlib.pyplot as plt
import pickle

In [15]:
IMG_SIZE = 256  # Resizing all images to 256x256
TRAIN_SPLIT = 0.8  # 80% training, 20% testing
OUTPUT_TRAIN_DIR = "processed_dataset/train"
OUTPUT_TEST_DIR = "processed_dataset/test"
SKETCH_DIR = r"C:\Users\Suyash Tambe\Desktop\sketch-photo\256x256\sketch\tx_000000000000"  # Use the correctly scaled sketches
PHOTO_DIR = r"C:\Users\Suyash Tambe\Desktop\sketch-photo\256x256\photo\tx_000000000000"  # Use the correctly scaled photos


In [16]:
os.makedirs(OUTPUT_TRAIN_DIR, exist_ok=True)
os.makedirs(OUTPUT_TEST_DIR, exist_ok=True)


In [17]:
class SketchToImageDataset(Dataset):
    def __init__(self, sketch_dir, photo_dir, transform=None):
        self.sketch_dir = sketch_dir
        self.photo_dir = photo_dir
        self.transform = transform
        self.categories = os.listdir(sketch_dir)
        self.data_pairs = []

        for category in self.categories:
            sketch_category_path = os.path.join(sketch_dir, category)
            photo_category_path = os.path.join(photo_dir, category)
            
            if not os.path.exists(photo_category_path):
                continue  # Skip categories without corresponding photos

            sketches = sorted(os.listdir(sketch_category_path))
            photos = sorted(os.listdir(photo_category_path))
            
            for sketch, photo in zip(sketches, photos):
                sketch_path = os.path.join(sketch_category_path, sketch)
                photo_path = os.path.join(photo_category_path, photo)
                self.data_pairs.append((sketch_path, photo_path))

    def __len__(self):
        return len(self.data_pairs)

In [6]:
import os
import shutil
import random

# Paths
TEST_SKETCH_DIR = r"C:\Users\Suyash Tambe\Desktop\sketch-photo\processed_dataset_1\test\sketch"
TEST_PHOTO_DIR = r"C:\Users\Suyash Tambe\Desktop\sketch-photo\processed_dataset_1\test\photo"
BACKUP_SKETCH_DIR = r"C:\Users\Suyash Tambe\Desktop\sketch-photo\backup_test\sketch"
BACKUP_PHOTO_DIR = r"C:\Users\Suyash Tambe\Desktop\sketch-photo\backup_test\photo"

# Ensure backup folders exist
os.makedirs(BACKUP_SKETCH_DIR, exist_ok=True)
os.makedirs(BACKUP_PHOTO_DIR, exist_ok=True)

# List all sketch & photo files
sketch_files = os.listdir(TEST_SKETCH_DIR)
photo_files = os.listdir(TEST_PHOTO_DIR)

# Extract base filenames (without extensions)
sketch_base_names = set(os.path.splitext(f)[0] for f in sketch_files)
photo_base_names = set(os.path.splitext(f)[0] for f in photo_files)

# Find common pairs
matching_pairs = list(sketch_base_names.intersection(photo_base_names))

# Shuffle and select half of them
random.shuffle(matching_pairs)
to_delete = matching_pairs[:len(matching_pairs) // 2]  # Half of the dataset

# Move files to backup
for base_name in to_delete:
    sketch_path = os.path.join(TEST_SKETCH_DIR, base_name + ".png")
    photo_path = os.path.join(TEST_PHOTO_DIR, base_name + ".jpg")

    backup_sketch_path = os.path.join(BACKUP_SKETCH_DIR, base_name + ".png")
    backup_photo_path = os.path.join(BACKUP_PHOTO_DIR, base_name + ".jpg")

    # Move files safely
    if os.path.exists(sketch_path):
        shutil.move(sketch_path, backup_sketch_path)
    if os.path.exists(photo_path):
        shutil.move(photo_path, backup_photo_path)

# Final count after deletion
remaining_sketches = len(os.listdir(TEST_SKETCH_DIR))
remaining_photos = len(os.listdir(TEST_PHOTO_DIR))

print(f"✅ Moved and deleted {len(to_delete)} sketch-photo pairs.")
print(f"📂 Backup saved in {BACKUP_SKETCH_DIR} and {BACKUP_PHOTO_DIR}.")
print(f"🔹 Remaining Sketches: {remaining_sketches}")
print(f"🔹 Remaining Photos: {remaining_photos}")

✅ Moved and deleted 158578 sketch-photo pairs.
📂 Backup saved in C:\Users\Suyash Tambe\Desktop\sketch-photo\backup_test\sketch and C:\Users\Suyash Tambe\Desktop\sketch-photo\backup_test\photo.
🔹 Remaining Sketches: 158578
🔹 Remaining Photos: 158578


In [5]:
import os

TEST_SKETCH_DIR = r"C:\Users\Suyash Tambe\Desktop\sketch-photo\processed_dataset_1\test\sketch"
TEST_PHOTO_DIR = r"C:\Users\Suyash Tambe\Desktop\sketch-photo\processed_dataset_1\test\photo"

# List files
sketch_files = os.listdir(TEST_SKETCH_DIR)
photo_files = os.listdir(TEST_PHOTO_DIR)

# Extract base filenames (removing extensions)
sketch_base_names = set([os.path.splitext(f)[0] for f in sketch_files])
photo_base_names = set([os.path.splitext(f)[0] for f in photo_files])

# Find common base names
common_base_names = sketch_base_names.intersection(photo_base_names)

# Print results
print(f"🔹 Total Sketch Files: {len(sketch_files)}")
print(f"🔹 Total Photo Files: {len(photo_files)}")
print(f"🔹 Matching Sketch-Photo Pairs: {len(common_base_names)}")

# Print first 10 matches
print(f"✅ Sample Matches: {list(common_base_names)[:10]}")

🔹 Total Sketch Files: 317156
🔹 Total Photo Files: 317156
🔹 Matching Sketch-Photo Pairs: 317156
✅ Sample Matches: ['305343', '71464', '286724', '306917', '49501', '201287', '27987', '305007', '233691', '2698']


In [7]:
import cv2
import numpy as np

def apply_clahe(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read in grayscale
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced_img = clahe.apply(img)
    return enhanced_img

# Example usage:
enhanced_sketch = apply_clahe(r"256x256\sketch\tx_000000000000\airplane\n02691156_58-1.png")
cv2.imwrite("enhanced_sketch.png", enhanced_sketch)  # Save the enhanced image


True

In [1]:
import os
import random
import shutil

# Current input folders (remaining images)
sketch_dir = r"processed_dataset_1/train/sketch"
photo_dir = r"processed_dataset_1/train/photo"

# Backup folders
backup_sketch_dir = r"processed_dataset_1/deleted_backup/sketch"
backup_photo_dir = r"processed_dataset_1/deleted_backup/photo"

# Ensure backup folders exist
os.makedirs(backup_sketch_dir, exist_ok=True)
os.makedirs(backup_photo_dir, exist_ok=True)

# Get all sketch filenames
all_filenames = sorted(os.listdir(sketch_dir))
num_total = len(all_filenames)
num_to_move = num_total // 2  # Delete half

# Randomly select half of the files
to_move = random.sample(all_filenames, num_to_move)

# Move files
for filename in to_move:
    sketch_path = os.path.join(sketch_dir, filename)
    photo_path = os.path.join(photo_dir, filename.replace('.png', '.jpg'))  # Adjust if needed

    sketch_backup_path = os.path.join(backup_sketch_dir, filename)
    photo_backup_path = os.path.join(backup_photo_dir, filename.replace('.png', '.jpg'))

    # Move sketch image
    if os.path.exists(sketch_path):
        shutil.move(sketch_path, sketch_backup_path)

    # Move photo image
    if os.path.exists(photo_path):
        shutil.move(photo_path, photo_backup_path)

print(f"✅ Moved another {num_to_move} sketch-photo pairs to 'deleted_backup' folder.")


✅ Moved another 79529 sketch-photo pairs to 'deleted_backup' folder.
